In [ ]:
pip install yfinance pandas numpy matplotlib statsmodels


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import coint

plt.style.use("seaborn-v0_8")


In [ ]:
# 2️⃣ Download Data from Yahoo Finance
tickers = ["RELIANCE.NS", "ONGC.NS"]

data = yf.download(
    tickers,
    start="2018-01-01",
    end="2024-12-01",
    interval="1d"
)["Close"]

In [ ]:
# Remove any missing data
data.dropna(inplace=True)
data.columns = ["RELIANCE", "ONGC"]

print("Data Downloaded Successfully:")
print(data.head())


In [ ]:
#  Visualize Prices
plt.figure(figsize=(12,5))
plt.plot(data.RELIANCE, label="RELIANCE")
plt.plot(data.ONGC, label="ONGC")
plt.legend()
plt.title("Adjusted Closing Prices (RELIANCE & ONGC)")
plt.show()

In [ ]:
#  Test for Cointegration
score, pvalue, _ = coint(data.RELIANCE, data.ONGC)
print(f"Cointegration p-value: {pvalue}")

if pvalue < 0.05:
    print(" Stocks are cointegrated. Proceeding with strategy.")
else:
    print(" Stocks are NOT cointegrated. Strategy may not work well.")

In [ ]:
#  Calculate Spread and Z-Score
spread = data.RELIANCE - data.ONGC
mean = spread.mean()
std = spread.std()
zscore = (spread - mean) / std

In [ ]:
# Plot Z-Score
plt.figure(figsize=(12,4))
plt.plot(zscore)
plt.axhline(0, color='black')
plt.axhline(1.5, color='red', linestyle='--')
plt.axhline(-1.5, color='red', linestyle='--')
plt.title("Z-Score of Spread")
plt.show()

# Generate Trading Signals
# Z > 1.5 -> Short RELIANCE, Long ONGC
# Z < -1.5 -> Long RELIANCE, Short ONGC
# Z around 0 -> Exit positions

positions = pd.DataFrame(index=data.index)
positions["RELIANCE"] = -zscore
positions["ONGC"] = zscore

In [ ]:
# Backtest Strategy
returns = data.pct_change().dropna()

strategy_returns = (
    positions.RELIANCE.shift(1) * returns.RELIANCE +
    positions.ONGC.shift(1) * returns.ONGC
)

cumulative_returns = (1 + strategy_returns).cumprod()

plt.figure(figsize=(12,4))
plt.plot(cumulative_returns)
plt.title("Pairs Trading Strategy Cumulative Returns")
plt.show()

In [ ]:
# Sharpe Ratio
sharpe = np.sqrt(252) * strategy_returns.mean() / strategy_returns.std()
print(f"Sharpe Ratio: {sharpe:.2f}")